# A09 Bagging
### Paola Figueroa
### 24/ Nov/2025

Entregable:
- AUC de Logitic Regression
- AUC de Bagging
- Comparación
- Conslusiones

In [2]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt import gp_minimize
from sklearn.metrics import roc_auc_score
from scipy.stats import norm
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier

In [3]:
data2=pd.read_csv("Default.csv")

In [4]:
data2['student_Yes'] = pd.get_dummies(data2['student'], drop_first=True)
X3 = data2[['balance', 'income', 'student_Yes']]
Y= data2['default']=='Yes'
X_multi = LogisticRegression()
X_multi.fit(X3, Y)

LogisticRegression()

In [5]:
# AUC Regresión Logística
y_probs_logreg = X_multi.predict_proba(X3)[:, 1]
auc_logreg = roc_auc_score(Y, y_probs_logreg)
print(f"AUC Regresión Logística: {auc_logreg}")

AUC Regresión Logística: 0.9495714810703948


In [10]:
n_bootstraps = 500
all_cols = X3.columns
pred_samples = np.zeros((n_bootstraps, len(Y)))
selected_cols_list = []
models = []
for i in range(n_bootstraps):

    selected_cols = np.random.choice(all_cols, size=2, replace=False)
    selected_cols_list.append(selected_cols)
    idx = np.random.choice(len(Y), size=len(Y), replace=True)
    X_sample = X3.iloc[idx][selected_cols]
    y_sample = Y[idx]

    model = DecisionTreeClassifier()
    model.fit(X_sample, y_sample)
    models.append(model)

for i in range(n_bootstraps):
    selected_cols = selected_cols_list[i]
    pred_samples[i, :] = models[i].predict_proba(X3[selected_cols])[:, 1]

bagged_pred = pred_samples.mean(axis=0)
auc_bagging = roc_auc_score(Y, bagged_pred)

print("AUC del Bagging:", auc_bagging)


AUC del Bagging: 1.0


In [ ]:
# tabla de comparación
coef_auc = np.array([0.9495714810703948, 1.0 ])
variables_auc = ['Regresión Logística','Bagging']

df_auc = pd.DataFrame({
    'AUC': coef_auc,
}, index=variables_auc)
print(df_auc)

                          AUC
Regresión Logística  0.949571
Bagging              1.000000


## Conclusiones
Como se puede ver, el AUC de Bagging es mayor que el de la regresión logística, lo cual significa que usar árboles de decisión captura patrones no lineales y complejos en los datos que la regresión logística no puede modelar. El AUC = 1.0 sugiere que el conjunto de árboles logra una separación casi perfecta entre clases, lo cual indica una discriminación superior. Sin embargo, un desempeño tan alto también puede ser señal de sobreajuste, (una p de pura chiripa), lo cual hace ruido pero checando la asignación de las variables no se ve ningún error. 